In [1]:
# Importando os módulos:
import pandas as pd
import folium
from folium import plugins, Map
import matplotlib.pyplot as plt
import collections
import datetime
import numpy as np
from API_google import reverse_geocode

In [2]:
# Leitura do arquivo base
df = pd.read_csv(r"dataset_v01.csv", delimiter=";")
df = df.rename(columns={'id': 'ID_USUARIO', 'dat_nsc': 'DATA_NASCIMENTO', 'renda': 'RENDA', 'renda_SPEC': 'RENDA_FIXA', 'latitude': 'LATITUDE', 'longitude': 'LONGITUDE', 'tipo_pessoa': 'TIPO_PESSOA'})

# Criando novas colunas CIDADE, ESTADO e LOGRADOURO
df['CIDADE'] = 'NaN'
df['ESTADO'] = 'NaN'
df['LOGRADOURO'] = 'NaN'

# Formatação das coordenadas, trocado as virgulas por ponto e removendo registros sem coordenadas
df['LATITUDE'] = df['LATITUDE'].str.replace(',','.')
df['LONGITUDE'] = df['LONGITUDE'].str.replace(',','.')
#df.dropna(subset=['LATITUDE'], inplace=True)

In [3]:
# Utilização da API Google Maps para inserção dos dados regionais, "CIDADE" e "ESTADO"
for index, row in df.iterrows():
    try:
        result = reverse_geocode(row['LATITUDE'], row['LONGITUDE'])
        df.loc[index,'CIDADE'] = result[0]
        df.loc[index,'ESTADO'] = result[1]
        df.loc[index,'LOGRADOURO'] = result[2]
        df.loc[index,'TIPO_PESSOA'] = result[3]
    except:
        pass

In [4]:
df

,ID_USUARIO,DATA_NASCIMENTO,RENDA,RENDA_FIXA,LATITUDE,LONGITUDE,TIPO_PESSOA,CIDADE,ESTADO,LOGRADOURO
0,d6c0cc1d-eb61-486a-b5c3-50725ef44d61,30/01/1981 00:00,"DE R$ 1.900,00 A R$ 3.800,00","3.104,00",-4.84758,-37.78439,CPF,Jaguaruana,Ceará,street_address
1,da0067e7-aa6b-4355-b373-ca17f6ac3b44,28/01/1978 00:00,"DE R$ 3.800,00 A R$ 7.600,00","5.703,00",-26.99011814,-48.63716523,CPF,Balneário Camboriú,Santa Catarina,street_address
2,2d729b8a-d53c-48e7-8bf5-33b021980c04,09/12/1954 00:00,"DE R$ 3.800,00 A R$ 7.600,00","3.901,00",-5.053671016,-42.81315203,CPF,Teresina,Piauí,street_address
3,62c0f1e6-36f2-4205-a134-232a58a6f8a5,28/08/1975 00:00,"DE R$ 788,00 A R$ 1.140,00","1.086,00",-15.059042,-48.163372,CNPJ,Mimoso de Goiás,Goiás,establishment
4,0cef9719-0aa3-49a2-adea-9c09c04de071,17/06/1957 00:00,"DE R$ 788,00 A R$ 1.140,00",836,-17.694658,-42.52228,CPF,Capelinha,Minas Gerais,street_address
...,...,...,...,...,...,...,...,...,...,...
9995,f2261fce-83c5-4f64-b145-85f46bcf18b1,08/10/1979 00:00,"DE R$ 1.900,00 A R$ 3.800,00","2.776,00",-11.60937194,-47.04750645,CPF,Porto Alegre do Tocantins,Tocantins,route
9996,bdaa7802-4650-44a4-93f7-952b72ec17a0,30/04/1980 00:00,"DE R$ 3.800,00 A R$ 7.600,00","4.099,00",-15.65049668,-47.79332059,CPF,Brasília,Distrito Federal,street_address
9997,e7b37cd5-714a-44d8-ad7f-4ed3aa6b0695,26/05/1976 00:00,"DE R$ 788,00 A R$ 1.140,00","1.052,00",-8.330678649,-36.13689703,CPF,São Caetano,Pernambuco,street_address
9998,167dbd7f-7ccd-4aad-a6b9-c3fb9d56c9cd,06/11/1985 00:00,"DE R$ 788,00 A R$ 1.140,00",958,NaN,NaN,CNPJ,NaN,NaN,NaN
